# Time domain reflectometry, measurement vs simulation

## Setup

In [ ]:
%matplotlib inline
import skrf
import numpy as np
import matplotlib.pyplot as plt
skrf.stylely()

## Load data

In [ ]:
#load all measurement and simulation data into a dictionary
meas = skrf.read_all_networks('tdr_measurement_vs_simulation/measurement/')
simu = skrf.read_all_networks('tdr_measurement_vs_simulation/simulation/')

# DUT
meas['P1-MSL_Stepped_140-P2'] = meas['P1-MSL_Stepped_140-P2'].extrapolate_to_dc(kind='linear', dc_sparam=[[0,1],[1,0]])
simu['MSL_Stepped_140'] = simu['MSL_Stepped_140'].extrapolate_to_dc(kind='linear', dc_sparam=[[0,1],[1,0]])
meas['P1-MSL_Short_50'] = meas['P1-MSL_Short_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[-1]])
simu['MSL_Short_50'] = simu['MSL_Short_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[-1]])
meas['P1-MSL_Open_50']  = meas['P1-MSL_Open_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[1]])
simu['MSL_Open_50']  = simu['MSL_Open_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[1]])
meas['P1-MSL_Load_50']  = meas['P1-MSL_Load_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[0]])
simu['MSL_Load_50']  = simu['MSL_Load_50'].extrapolate_to_dc(kind='linear', dc_sparam=[[0]])

## comparison
### Stepped line

In [ ]:
plt.figure()
plt.title('Stepped line')
meas['P1-MSL_Stepped_140-P2'].s11.plot_z_time_step(window='hamming', pad=5000, label='measurement')
simu['MSL_Stepped_140'].s11.plot_z_time_step(window='hamming', pad=5000, label='simulation')
plt.xlim((-1e-9, 3e-9))

### Short, open, load

In [ ]:
plt.figure()
plt.title('Short, open, load')
meas['P1-MSL_Short_50'].plot_s_time_step(window='hamming', label='short meas')
meas['P1-MSL_Open_50'].plot_s_time_step(window='hamming', label='open meas')
meas['P1-MSL_Load_50'].plot_s_time_step(window='hamming', label='load meas')
simu['MSL_Short_50'].plot_s_time_step(window='hamming', label='short simu')
simu['MSL_Open_50'].plot_s_time_step(window='hamming', label='open simu')
simu['MSL_Load_50'].plot_s_time_step(window='hamming', label='load simu')
plt.xlim((-1e-9, 3e-9))

## Simulation

In [ ]:
freq = skrf.F(1,10e3,10000, 'mhz')

# Mechanical section in meter
W   = 3.00e-3
H   = 1.51e-3
T   = 50e-6
# epsilon_r and tand at 1.0 GHz.
f_epr_tand = 1e9
epr = 4.369
tand = 0.0157 

# 50 ohm segment
MSL1 = skrf.media.MLine(frequency=freq, z0=50, w=W, h=H, t=T,
        ep_r=epr, mu_r=1, rho=1.712e-8, tand=tand, rough=0.15e-6,
        f_low=1e3, f_high=1e12, f_epr_tand=f_epr_tand,
        diel='djordjevicsvensson', disp='kirschningjansen')

# Capacitive segment
MSL2 = skrf.media.MLine(frequency=freq, z0=50, w=8.0e-3, h=H, t=T,
        ep_r=epr, mu_r=1, rho=1.712e-8, tand=tand, rough=0.15e-6,
        f_low=1e3, f_high=1e12, f_epr_tand=f_epr_tand,
        diel='djordjevicsvensson', disp='kirschningjansen')

# Inductive segment
MSL3 = skrf.media.MLine(frequency=freq, z0=50, w=1.0e-3, h=H, t=T,
        ep_r=epr, mu_r=1, rho=1.712e-8, tand=tand, rough=0.15e-6,
        f_low=1e3, f_high=1e12, f_epr_tand=f_epr_tand,
        diel='djordjevicsvensson', disp='kirschningjansen')

step_simu = MSL1.line(50e-3, 'm', embed=True, z0=MSL1.Z0_f) \
      ** MSL2.line(20e-3, 'm', embed=True, z0=MSL2.Z0_f) \
      ** MSL3.line(20e-3, 'm', embed=True, z0=MSL3.Z0_f) \
      ** MSL1.line(50e-3, 'm', embed=True, z0=MSL1.Z0_f)
step_simu.name = 'MSL_Stepped_140'

short_simu = MSL1.delay_short(50e-3, 'm', embed=True, z0=MSL1.Z0_f)
short_simu.name = 'MSL_Short_50'
open_simu = MSL1.delay_open(50e-3, 'm', embed=True, z0=MSL1.Z0_f)
open_simu.name = 'MSL_Open_50'
load_simu = MSL1.delay_load(0, 50e-3, 'm', embed=True, z0=MSL1.Z0_f)
load_simu.name = 'MSL_Load_50'
    
write_data = False
if write_data:
    step_simu.write_touchstone(dir='tdr_measurement_vs_simulation/simulation/')
    short_simu.write_touchstone(dir='tdr_measurement_vs_simulation/simulation/')
    open_simu.write_touchstone(dir='tdr_measurement_vs_simulation/simulation/')
    load_simu.write_touchstone(dir='tdr_measurement_vs_simulation/simulation/')